In [11]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
from google.cloud import vision
import io
import time
import os

from tqdm import tqdm
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join('.', 'gcp/googlecreds.json')

text_list = []


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            class_name = member[0].text
            if class_name == 'keyvalue':
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[4][0].text),
                         int(member[4][1].text),
                         int(member[4][2].text),
                         int(member[4][3].text)
                         )
                xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def getTextGroupData(imagePath):
    vertexList = []
    client = vision.ImageAnnotatorClient()
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    bool = False
    complete_text = ''
    for text in texts:
        vertices = text.bounding_poly.vertices
        if bool:
            vertexList.append((text.description, vertices))
        else:
            complete_text = text.description
            bool = True

    if response.error.message:
        raise Exception(
            '{}\n'.format(response.error.message))
    return vertexList



def get_text_single_bbox(vertexList, bbox):
    text = ''
    boundingBox = bbox

    for vertexText in vertexList:
        vertex = vertexText[1]
        x1 = vertex[0].x
        y1 = vertex[0].y
        x2 = vertex[2].x
        y2 = vertex[2].y
        height = y2 - y1
        width = x2 - x1

        if (not (max(x1, boundingBox[0]) + width / 3 > min(x2, boundingBox[2]))) and \
                (not (max(y1, boundingBox[1]) + height / 3 > min(y2, boundingBox[3]))):
            text += vertexText[0] + ' '
    text_list.append(text)
    return text


In [16]:
def main():
    base_path = 'C:/Users/KartikayGupta/NER/new_data/ci/'
    xml_df = xml_to_csv(base_path)
    xml_df.to_excel('AN+BL_annotations.xlsx')
    xml_df = pd.read_excel('AN+BL_annotations.xlsx', engine='openpyxl')

    final_df = pd.DataFrame()
    files = xml_df['filename'].unique()
    i = 0
    for file in tqdm(files):
        start = time.time()
        df = xml_df[xml_df['filename'] == file]
        if not os.path.exists(base_path + file):
            continue
        vertexList = getTextGroupData(base_path + file)
        for index, row in df.iterrows():
            bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
            text = get_text_single_bbox(vertexList, bbox)
            final_df.loc[i, 'filename'] = file
            final_df.loc[i, 'text'] = text
            final_df.loc[i, 'label'] = df.loc[index, 'class']
            i = i+1
        print(file + ' --> done')
        print(time.time()-start)
    #final_df.to_excel('AN+BL-not-seperated-keyvalues.xlsx')


if __name__ == '__main__':
    main()


  4%|▍         | 1/25 [00:02<01:01,  2.56s/it]

100678791-699489342-CIPLUPDATE-1.png --> done
2.558120012283325


  8%|▊         | 2/25 [00:04<00:50,  2.21s/it]

101352736-1017484529-XLS-CIPLforNBMZ136913-1.png --> done
1.9627797603607178


 12%|█▏        | 3/25 [00:06<00:47,  2.18s/it]

...

438387920-438387759-CopyofInvoice-packinglist-86882-1.png --> done
2.034600019454956


 96%|█████████▌| 24/25 [01:12<00:02,  2.87s/it]

56776715-1732358103-comminv21012021-1.png --> done
2.3341317176818848


100%|██████████| 25/25 [01:18<00:00,  3.15s/it]

7375694-882004595-PAINANI530411MAR22-5.png --> done
5.976384401321411


In [17]:
print(len(text_list))

1113


In [18]:
with open('final_data.txt', 'w' , encoding="utf-8") as f:
    for line in text_list:
        f.write(line)
        f.write('\n')